# Deepfake Generation using a DCGAN

## Tristan Breetz - CSC494

### Import TensorFlow and other libraries

In [1]:
import tensorflow as tf

In [2]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time

from IPython import display

## Import dataset

In [3]:
BATCH_SIZE = 128
IMG_HEIGHT = 64
IMG_WIDTH = 64
DIR = './img_align_celeba'
STEPS_PER_EPOCH = np.ceil(202599/BATCH_SIZE)

image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
print(glob.glob(DIR + '*.jpg'))
train_data_gen = image_generator.flow_from_directory(directory=DIR,
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH))

[]
Found 202599 images belonging to 202599 classes.


In [4]:
#Model Parameters
EPOCHS = 50
NOISE_DIM = 100 
LEARNING_RATE = 0.0002

### The Generator Model

Based on DCGAN architecture
    Radford, Metz, Chintala (2016)

In [5]:
def make_generator_model():
    model = tf.keras.Sequential()
    
    model.add(layers.Dense(4*4*1024, use_bias=False, input_shape=(NOISE_DIM,), kernel_initializer='random_normal',
                bias_initializer='random_normal'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((4, 4, 1024)))
    assert model.output_shape == (None, 4, 4, 1024) #None corresponds to the batch size

    model.add(layers.Conv2DTranspose(512, (5, 5), strides=(2, 2), 
                                     padding='same',
                                     use_bias=False,
                                     kernel_initializer='random_normal',
                                     bias_initializer='random_normal'))
    assert model.output_shape == (None, 8, 8, 512)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(256, (5, 5), strides=(2, 2), 
                                     padding='same',
                                     use_bias=False,
                                     kernel_initializer='random_normal',
                                     bias_initializer='random_normal'))
    assert model.output_shape == (None, 16, 16, 256)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same',
                                     use_bias=False,
                                     kernel_initializer='random_normal',
                                     bias_initializer='random_normal'))
    assert model.output_shape == (None, 32, 32, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same',
                                     use_bias=False, activation='tanh',
                                     kernel_initializer='random_normal', 
                                     bias_initializer='random_normal'))
    assert model.output_shape == (None, 64, 64, 3)

    return model

### The Discriminator Model

The discriminator is a CNN image classifier (real/fake binary classification).

In [6]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(256, (5, 5), strides=(2, 1), padding='same',
                                     input_shape=[64, 64, 3]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))
    model.add(layers.LeakyReLU())
    
    return model

## Define the loss and optimizers

Define loss functions and optimizers for both models.


In [7]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

### Discriminator loss
Measurement of discriminator performance, the lower the loss, the higher its ability to determine a real image from a fake.

In [8]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

### Generator loss
The generator loss function can be thought of "If the discriminator is doing poorly, I must be doing well."

In [9]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

The discriminator and the generator optimizers are different since we will train two networks separately.

In [10]:
generator_optimizer = tf.keras.optimizers.Adam(
    learning_rate=LEARNING_RATE, beta_1=0.5, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam'
)

discriminator_optimizer = tf.keras.optimizers.Adam(
    learning_rate=LEARNING_RATE, beta_1=0.5, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam'
)

In [11]:
generator = make_generator_model()
discriminator = make_discriminator_model()

## Define the training loop


In [12]:
gen_loss_history = []
discrim_loss_history = []

In [13]:
#Training function, where loss for both the discriminator and generator modes are determined, the gradients are then applied to the weights
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, NOISE_DIM])
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(train_data_gen.next()[0], training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        discrim_loss = discriminator_loss(real_output, fake_output)
        
    #These two lines apply gradient descent function on the traininable layers
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(discrim_loss, discriminator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    return gen_loss, discrim_loss

In [14]:
def train(dataset, epochs): #returns loss history after training
    gen_loss_hist = []
    discrim_loss_hist = []
    for epoch in range(epochs):
        epoch = epoch
        epoch_start = time.time()
        #Each epoch generate 16 sample images and save them to disk
        seed = tf.random.normal([16,NOISE_DIM])
        display.clear_output(wait= True)
        generate_and_save_images(generator, epoch, seed)

        #Complete the training steps for each batch of images
        step_count = 0
        for image_batch in dataset:
            gen_loss, discrim_loss = train_step(image_batch)
            gen_loss_hist.append(gen_loss.numpy())
            discrim_loss_hist.append(discrim_loss.numpy())
    
        # Save the model every epoch
        generator.save(f'models/lr-.{LEARNING_RATE}-bs-{BATCH_SIZE}-epoch-{epoch}-face_gen.h5')
        discriminator.save(f'models/lr-.{LEARNING_RATE}-bs-{BATCH_SIZE}-epoch-{epoch}-face_discrim.h5')
    return gen_loss_hist, discrim_loss_hist

**Generate and save images**


In [15]:
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    fig = plt.figure(figsize=(64,64))
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(np.asarray((predictions[i, :, :,] * 255)).astype(np.uint8))
        plt.axis('off')
    plt.savefig(f'gan_images/lr-{LEARNING_RATE}-bs-{BATCH_SIZE}-epoch-{epoch}.png')
    plt.show()

## Train the models

In [ ]:
train_data_gen.batch_size = BATCH_SIZE
train(train_data_gen.next()[0], EPOCHS)


.## Save the final models

In [ ]:
generator.save(f'models/lr-.{LEARNING_RATE}-bs-{BATCH_SIZE}-FINAL-face_gen.h5')
discriminator.save(f'models/lr-.{LEARNING_RATE}-bs-{BATCH_SIZE}-FINAL-face_discrim.h5')